In [1]:
import PIL
import subprocess
import re
import os
import time
import io

In [2]:
def capture_img():
    subprocess.call('.\\adb\\adb shell /system/bin/screencap -p /sdcard/screenshot.png')
    time.sleep(0.1)
    subprocess.call('.\\adb\\adb pull /sdcard/screenshot.png %s' % './tmp/screenshot.png')
    time.sleep(0.1)

    im = PIL.Image.open('./tmp/screenshot.png')
    im = im.convert('RGB').crop((0,300,1080,600))
#     im.thumbnail((150,720))
    im.save('./tmp/screenshot.jpg')

    with open('./tmp/screenshot.jpg', 'rb') as fp:
        return fp.read()

In [3]:
%run aip_key.py

In [4]:
from aip import AipOcr

client = AipOcr(APP_ID, API_KEY, SECRET_KEY)

In [5]:
def search_ans():
    image = capture_img()

    print('Parsing pic...')
    rst = client.basicGeneral(image)

    question = ''.join([p['words'] for p in rst['words_result']])

    if question != None and question != '':
#         question = question.split('.')[1]

        qs = re.split(r"\.|,|。|，",question)
        if len(qs) > 1:
            question = ''.join(qs[1:])
        print('question:', question)
                
#         os.system('start www.baidu.com/s?wd=%s' % question)
        os.system('start www.google.com/search?q=%s' % question)

    else:
        print('No question found!')

In [ ]:
while True:
    ipt = input()
    if ipt == 'q':
        break
    else:
        search_ans()


Parsing pic...
question: “岁寒三友”不包括下面哪项?

Parsing pic...
question: 我国著名的革命根据地延安位于哪个省?

Parsing pic...
question: 以下哪个运动员和林丹是同个项目的?

Parsing pic...
question: 苏东坡是个吃货但是宋朝的他吃不到下列哪种食物?

Parsing pic...
question: 钢笔写字时常会碰到堵笔现象请问墨水残留物主要是由下列哪个化学式组成的?

Parsing pic...
question: “雄关漫道真如铁"这句诗中的“漫道”是什么意思?
